# Anomaly Detection (Trades) - Model Generation

We will use the [Support Vector Data Description (SVDD)](https://go.documentation.sas.com/?cdcId=pgmsascdc&cdcVersion=9.4_3.5&docsetId=casactml&docsetTarget=casactml_svdatadescription_details.htm&locale=en) algorithm, packaged in SAS Visual Data Mining and Machine Learning (VDMML), and deploy it using SAS Event Stream Processing ESPPy module.The CAS session is used to invoke Deep Learning to train a new model. to detect outliers in real time using streaming data.


### 0. Setup the Environment

First, import the necessary packages to run this notebook.

In [172]:
import os
import pandas as pd
import swat
import getpass

### 1. Start a SAS Viya CAS Session

The CAS session is used to invoke SAS Visual Data Mining and Machine Learning (VDMML) to train a new model.

In [173]:
os.environ["CAS_CLIENT_SSL_CA_LIST"] = "/opt/sas/viya/config/etc/SASSecurityCertificateFramework/cacerts/trustedcerts.pem"
password=getpass.getpass();

cashost='frasepviya35smp'
casport=5570
sess = swat.CAS(cashost, casport,'viyademo01',password)


sess.loadactionset(actionset="table")

 ······


NOTE: Added action set 'table'.


[actionset]

 'table'

+ Elapsed: 0.000309s, sys: 0.000283s, mem: 0.221mb

### 2. Data Preparation

Define the data to be used during model generation

In [174]:
# Name the existing CAS table to be used
indata = 'STREAMTOTALCOST_TRAIN'

if not sess.table.tableExists(name=indata,caslib="public").exists:
    tbl = sess.table.loadTable(caslib="public", path=indata+".sashdat", casout={"name":indata,"caslib":"public", "replace":True})

castbl = sess.CASTable(name=indata, caslib="public")

NOTE: Cloud Analytic Services made the file STREAMTOTALCOST_TRAIN.sashdat available as table STREAMTOTALCOST_TRAIN in caslib public.


In [175]:
sess.tableinfo(caslib="public")

,Name,Rows,Columns,IndexedColumns,Encoding,CreateTimeFormatted,ModTimeFormatted,AccessTimeFormatted,JavaCharSet,CreateTime,...,Repeated,View,MultiPart,SourceName,SourceCaslib,Compressed,Creator,Modifier,SourceModTimeFormatted,SourceModTime
0,STREAMTOTALCOST_TRAIN,137100,9,0,utf-8,2021-03-08T22:03:33+01:00,2021-03-08T22:03:33+01:00,2021-03-08T22:03:33+01:00,UTF8,1.930857e+09,...,0,0,0,STREAMTOTALCOST_TRAIN.sashdat,Public,0,viyademo01,,2021-03-08T19:40:19+01:00,1.930848e+09
1,STREAMTOTALCOST,37700,11,0,utf-8,2021-03-08T16:29:32+01:00,2021-03-08T16:35:09+01:00,2021-03-08T21:40:42+01:00,UTF8,1.930837e+09,...,0,0,0,,,0,viyademo01,,,NaN
2,STREAMTRADES,37700,7,0,utf-8,2021-03-08T16:29:32+01:00,2021-03-08T16:35:09+01:00,2021-03-08T16:44:39+01:00,UTF8,1.930837e+09,...,0,0,0,,,0,viyademo01,,,NaN
3,TEST_1_UNDEFINED_2021_03_08_21_40_35_OUTPUT,37700,11,0,utf-8,2021-03-08T21:40:42+01:00,2021-03-08T21:40:42+01:00,2021-03-08T21:41:43+01:00,UTF8,1.930855e+09,...,0,0,0,,,0,viyademo01,,,NaN


In [176]:
castbl.describe()
list(castbl.columns)

['_opcode',
 'tradeID*',
 'security',
 'quantity',
 'price',
 'totalCost',
 'traderID',
 'time',
 'name']

### 4. Import SVDD action set

In [177]:
sess.loadactionset('svdd')

NOTE: Added action set 'svdd'.


[actionset]

 'svdd'

+ Elapsed: 0.00261s, user: 0.00065s, sys: 0.00193s, mem: 0.23mb

### 5. Train SVDD Model

SVDD algorithm is a one-class classification technique that is useful in applications where data that belongs to one class is abundant, but data about any other class is scarce or missing. Fraud detection, equipment health monitoring, and process control are some examples of application areas where the majority of the data belong to one class.

In its simplest form, an SVDD model is obtained by building a minimum-radius hypersphere around the one-class training data. The hypersphere provides a compact spherical description of the training data. This training data description can be used to determine whether a new observation is similar to the training data observations. The distance from any new observation to the hypersphere center is computed and compared with the hypersphere radius. If the distance is more than the radius, the observation is designated as an outlier. Using kernel functions in SVDD formulation provides a more flexible description of training data. Such description is nonspherical and conforms to the geometry of the data. PROC SVDD implements only the flexible data description.

We are using SVDD based K-charts to determine anomalous behavior in AHUs. K-chart is a nonparametric multivariate control chart that is used for statistical process control and can also be used for monitoring equipment health and operating data. It is implemented in two phases: In phase 1, observations from normal operations of the process are collected and are used to train a SVDD model and obtain the threshold r-square value.

In [178]:
# Phase 1: Model Training 
# Run svDataDescription.svddTrain action set on Training data 
sess.svDataDescription.svddTrain(tunemethod="MEAN2",
                         solver='actset',
                         inputs=[{"name":"totalcost"}],
                         id=['_opcode','tradeID*','security','quantity','price','totalCost','traderID','time','name'],
                         savestate = {"name": "trade_outliers_svdd", "replace":True},
                         table={"caslib":"public","name":"STREAMTOTALCOST_TRAIN"})

NOTE: Using Active Set Solver.
NOTE: SVDDTRAIN runs with default maxtime of 1800 seconds.
NOTE: Beginning data reading...
NOTE: Data reading complete.
NOTE: Starting processing for Model 1 (with bandwidth=65582.063983).
NOTE: Ending processing for Model 1 (with bandwidth=65582.063983).
NOTE: Output generation complete.
NOTE: Beginning save state generation...
NOTE: 6781 bytes were written to the table "trade_outliers_svdd" in the caslib "CASUSER(viyademo01)".
NOTE: Save state generation complete.


[NObs]

 Number of Observations
 
    RowId                         Type         N
 0  NREAD  Number of Observations Read  137100.0
 1  NUSED  Number of Observations Used  137100.0

[ModelInfo]

 Model Information
 
        RowId                   Description          Value        nValue
 0  OPTMETHOD           Optimization Method     Active Set           NaN
 1    KERTYPE                   Kernel Type            RBF           NaN
 2         BW          RBF Kernel Bandwidth   65582.063983  65582.063983
 3       TUNE    Bandwidth Selection Method  Modified Mean           NaN
 4   RELSCALE      Bandwidth Relative Scale   1.3722162637      1.372216
 5       FRAC     Expected Outlier Fraction           1E-6      0.000001
 6     OPTTOL        Optimization Tolerance         0.0001      0.000100
 7   NINTVARS  Number of Interval Variables              1      1.000000
 8   NNOMVARS   Number of Nominal Variables              0      0.000000

[TrainingResults]

 Training Results
 
     RowId                            Description      Value
 0     NSV              Number of Support Vectors  17.000000
 1    NSVB  Number of Support Vectors on Boundary  17.000000
 2  NDROBS         Number of Dropped Observations   0.000000
 3  THRESH               Threshold R Square Value   0.854967
 4     C_R                   Constant (C_r) Value   0.145033
 5   RTIME                     Run Time (seconds)   0.489003
 6    BCAL   Bandwidth Calculation Time (seconds)   0.000049

[OptSummary]

 Optimization Summary
 
        RowId           Description         Value    nValue
 0     NITERS  Number of Iterations             1  1.000000
 1        OBJ       Objective Value  0.1450334578  0.145033
 2      INFEA         Infeasibility  0.0000955614  0.000096
 3  OPTSTATUS   Optimization Status       Optimal       NaN
 4      DEGEN            Degenerate            No  0.000000

[StatusTable]

 Status Table
 
      RowId    Model   Status
 0  Model_1  Model 1  Success

+ Elapsed: 0.6s, user: 0.644s, sys: 0.392s, mem: 843mb

### 6. Score SVDD Model using ASTORE

This model of normal operations is then operationalized in phase 2 for anomaly detection. For each new observation, its distance value is computed and compared to the threshold r-square value. Observations for which distance value > threshold r-square generally indicate something abnormal in the process.

Load astore action set

In [179]:
sess.loadactionset('astore')

NOTE: Added action set 'astore'.


[actionset]

 'astore'

+ Elapsed: 0.000696s, sys: 0.000679s, mem: 0.238mb

In [180]:
sess.score(
     table={"name":'STREAMTOTALCOST_TRAIN',"caslib":'public'},
     out='STREAMTOTALCOST_TRAIN_scored',
     rstore='trade_outliers_svdd'
    )

[OutputCasTables]

                 casLib                          Name    Rows  Columns                                           casTable
 0  CASUSER(viyademo01)  STREAMTOTALCOST_TRAIN_scored  137100       11  CASTable('STREAMTOTALCOST_TRAIN_scored', casli...

[Timing]

 Task Timing
 
                  Task   Seconds   Percent
 0   Loading the Store  0.000079  0.002107
 1  Creating the State  0.002190  0.058472
 2             Scoring  0.034938  0.932776
 3               Total  0.037456  1.000000

+ Elapsed: 0.0393s, user: 0.166s, sys: 0.0341s, mem: 54.3mb

### 7. Generate an Analytic Store File

This ASTORE file can then be used in ESPPy for real-time anomaly detection.

In [181]:
#store=sess.download(rstore='svdd_ahu')
#with open('../ahu_svdd.astore','wb') as file:
#   file.write(store['blob'])

In [182]:
#sess.astore.describe(rstore='svdd_ahu')

In [183]:
sess.tableinfo()

,Name,Rows,Columns,IndexedColumns,Encoding,CreateTimeFormatted,ModTimeFormatted,AccessTimeFormatted,JavaCharSet,CreateTime,...,Repeated,View,MultiPart,SourceName,SourceCaslib,Compressed,Creator,Modifier,SourceModTimeFormatted,SourceModTime
0,TRADE_OUTLIERS_SVDD,1,2,0,utf-8,2021-03-08T22:03:34+01:00,2021-03-08T22:03:34+01:00,2021-03-08T22:03:34+01:00,UTF8,1.930857e+09,...,0,0,0,,,0,viyademo01,,,NaN
1,STREAMTOTALCOST_TRAIN_SCORED,137100,11,0,utf-8,2021-03-08T22:03:34+01:00,2021-03-08T22:03:34+01:00,2021-03-08T22:03:34+01:00,UTF8,1.930857e+09,...,0,0,0,,,0,viyademo01,,,NaN


In [184]:
castbls = sess.CASTable("STREAMTOTALCOST_TRAIN_scored")
castbls.head(10)

,_opcode,tradeID*,security,quantity,price,totalCost,traderID,time,name,_SVDDDISTANCE_,_SVDDSCORE_
0,ins,91782,AOL,1612.0,110.6620,178387.1440,10004.0,2021-03-08 14:50:26.856708,Joe Bloggs,0.854468,-1.0
1,ins,91783,SAP,3322.0,195.5200,649517.4400,10004.0,2021-03-08 14:50:26.856710,Joe Bloggs,0.855090,1.0
2,ins,91784,TPC,1032.0,127.3360,131410.7520,10005.0,2021-03-08 14:50:26.856712,Jane Bloggs,0.854777,-1.0
3,ins,91785,TPC,1490.0,150.0680,223601.3200,10005.0,2021-03-08 14:50:26.856713,Jane Bloggs,0.855144,1.0
4,ins,91786,LQD,1032.0,127.3420,131416.9440,10005.0,2021-03-08 14:50:26.856714,Jane Bloggs,0.854777,-1.0
5,ins,91787,TPC,2162.0,183.4100,396532.4200,10005.0,2021-03-08 14:50:26.856716,Jane Bloggs,0.854966,-1.0
6,ins,91788,AOL,2131.0,181.8990,387626.7690,10005.0,2021-03-08 14:50:26.856717,Jane Bloggs,0.854971,1.0
7,ins,91789,SAP,2101.0,27.3454,57452.6854,10005.0,2021-03-08 14:50:26.856718,Jane Bloggs,0.844812,-1.0
8,ins,91790,TPC,2071.0,178.8760,370452.1960,10005.0,2021-03-08 14:50:26.856722,Jane Bloggs,0.854982,1.0
9,ins,91791,XOM,1582.0,154.6390,244638.8980,10004.0,2021-03-08 14:50:26.857299,Joe Bloggs,0.855085,1.0


### 8. Register model in model respository

Terminate the CAS session.

In [185]:
from sasctl import Session
from sasctl.tasks import register_model
from sasctl.services import model_repository,projects

astore = sess.CASTable('trade_outliers_svdd')

with Session('http://frasepviya35smp', 'viyademo01', password):
   try:
      existingProj=model_repository.get_project('Trade_ML002')
      model_repository.delete_project(existingProj)
   finally:
     model_repository.create_project('Trade_ML002','Public')
     register_model(astore, 'outlier_svdd002', 'Trade_ML002')

NOTE: Added action set 'astore'.
NOTE: Cloud Analytic Services saved the file _38A18B651AB1410ABB9080919.sashdat in caslib ModelStore.


### 9. Cleanup Your Project

Terminate the CAS session.

In [186]:
sess.close()